In [1]:
# LME_Model.ipynb - Linear Mixed Effect model for picking genes associated with experimental factors

# Version: N/A (8/23/2018)

# Niu Du (dniu [at] jcvi.org)
# J. Craig Venter Institute (JCVI)
# La Jolla, CA USA

# User environments
# python 3.5.4 
# pandas 0.23.1
# numpy 1.12.1
# read_dataframe - Function custom defined
# rpy2 2.9.4
# R 3.4.1
# lme4 1.1-17
# lmerTest 3.0-1
# lawstat 3.2

In [10]:
import pandas as pd
import numpy as np
import rpy2
import time
import os
import warnings
warnings.filterwarnings("ignore")

In [13]:
def Model_test(Fixed_factors,metadata,count_table,Sample_list): 
    
    %load_ext rpy2.ipython
    %R require(lme4);
    %R require(lmerTest);
    %R require(lawstat);
    
    # This function allows the test of mixed effect model for 3-4 factors with one random factor (family), 
    # and output the following parameters of linear model for each gene and fixed factor;
    # 1. Coefficient 2. Error 3. t value 4. p value (< threshold for significant correlation) 5. Heteroscedasticity of linear regression 
    # Fixed_factors - list of column names in metadata file;
    # metadata - pandas dataframe, sample ids in index and factors in columns;
    # count_table - pandas dataframe, samples in index and genes in colums;
    # Sample_list - list of ids that are selected for model test;

    Fixed_list = ['Intercept']+Fixed_factors
    list_0 = ['Coeff_'+x for x in Fixed_list]
    list_a = ['Err_'+ x for x in Fixed_list]
    list_b = ['t_value_'+ x for x in Fixed_list]
    list_c = ['Pr>|t|_'+ x for x in Fixed_list]
    list_d = ['Levene_'+x for x in Fixed_factors]
    Output_list = list_0+list_a+list_b+list_c+list_d
    
    Stat_result = pd.DataFrame(index = Output_list)
    df = metadata.loc[Sample_list].dropna()[Fixed_factors+['family']]
    df_counts = count_table.loc[df.index] 
    
    if len(Fixed_factors) == 3:
        
        for gene_ids in df_counts.columns:
            counts = df_counts[[gene_ids]]
            %R -i df
            %R -i counts
            %R mb = lmer(counts[[1]]~df[[1]]+df[[2]]+df[[3]]+(1|df[['family']]));
            %R stat = c(coef(summary(mb)), levene.test(counts[[1]],df[[1]])$p.value,levene.test(counts[[1]],df[[2]])$p.value,levene.test(counts[[1]],df[[3]])$p.value);
            %R -o stat
            Stat_result[gene_ids] = list(stat)[:8]+list(stat)[-11:]
        
    elif len(Fixed_factors) == 4:
        
        for gene_ids in df_counts.columns:
            counts = df_counts[[gene_ids]]
            %R -i df
            %R -i counts
            %R mb = lmer(counts[[1]]~df[[1]]+df[[2]]+df[[3]]+df[[4]]+(1|df[['family']]));
            %R stat = c(coef(summary(mb)), levene.test(counts[[1]],df[[1]])$p.value,levene.test(counts[[1]],df[[2]])$p.value,levene.test(counts[[1]],df[[3]])$p.value,levene.test(counts[[1]],df[[4]])$p.value);
            %R -o stat
            Stat_result[str(gene_ids)] = list(stat)[:10]+list(stat)[-14:]
        
    return Stat_result

In [4]:
metadata = pd.read_csv('caries_metadata.tsv',sep='\t').set_index('SubjectID')

In [ ]:
df_caries = read_dataframe('caries.fpkm.tsv');

In [ ]:
Result = Model_test(['Age', 'Sex','center','caries'],metadata,df_caries[df_caries.columns],list(df_caries.index))

In [20]:
Result.to_pickle('Model_results/fpkm_121418.pkl')